In [1]:
%matplotlib inline
%load_ext nb_black

2022-04-17 14:34:05,770 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/Grammar.txt
2022-04-17 14:34:05,795 - blib2to3.pgen2.driver - INFO - Writing grammar tables to /Users/juanazuero/Library/Caches/black/20.8b1/Grammar3.8.0.final.0.pickle
2022-04-17 14:34:05,796 - blib2to3.pgen2.driver - INFO - Writing failed: [Errno 2] No such file or directory: '/Users/juanazuero/Library/Caches/black/20.8b1/tmp57ia94s_'
2022-04-17 14:34:05,797 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/PatternGrammar.txt
2022-04-17 14:34:05,799 - blib2to3.pgen2.driver - INFO - Writing grammar tables to /Users/juanazuero/Library/Caches/black/20.8b1/PatternGrammar3.8.0.final.0.pickle
2022-04-17 14:34:05,800 - blib2to3.pgen2.driver - INFO - Writing failed: [Errno 2] No such file or directory: '/Users/juanazuero/Library/Caches/black/20.8b1/tmpl1

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import time


<IPython.core.display.Javascript object>

In [3]:
def unixTimeMillis(dt):
    """ Convert datetime to unix timestamp """
    return int(time.mktime(dt.timetuple()))


def getMarks(start, end, Nth=100):
    """Returns the marks for labeling.
    Every Nth value will be used.
    """
    daterange = pd.date_range(start=start, end=end, freq="Y")
    result = {
        unixTimeMillis(date + pd.Timedelta(value=1, unit="D")): str(
            (date + pd.Timedelta(value=1, unit="D")).strftime("%Y")
        )
        for date in daterange
    }
    return result


<IPython.core.display.Javascript object>

In [46]:
from datetime import date
from black import main
import pandas as pd
import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table
from kedro.framework.startup import bootstrap_project
from pathlib import Path
from kedro.framework.session import KedroSession
import plotly.express as px
from sklearn import metrics

# app
app = dash.Dash(__name__)
server = app.server
# Constants
SEASONALITIES = [
    "Yearly seasonality",
    "Weekly seasonality",
    "Dayly seasonalities",
    "Holidays",
]
LAYOUT_GRAPHS = {
    "paper_bgcolor": "rgba(255,255,255,100)",
    "plot_bgcolor": "rgba(255,255,255,100)",
}
XAXES_CONFIG = {
    "showgrid": True,
    "gridwidth": 1,
    "gridcolor": "lightgrey",
}
YAXES_CONFIG = {
    "showgrid": True,
    "gridwidth": 1,
    "gridcolor": "lightgrey",
}
YAXES_CONFIG_2 = {
    "zeroline": True,
    "zerolinewidth": 2,
    "zerolinecolor": "black",
}
DAY_OF_WEEK = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}
# Load data
forecast_df = catalog.load("forecast_system")
dayly_seasonalities = catalog.load("dayly_seasonalities_system")
weekly_seasonality = catalog.load("weekly_seasonality_system")
yearly_seasonality = catalog.load("yearly_seasonality_system")
holidays_ser = catalog.load("holidays_ser")
# Preparations
min_time = forecast_df["ds"].min()
max_time = forecast_df["ds"].max()
holidays_df = pd.DataFrame()
holidays_df["Holiday"] = list(holidays_ser.keys())
holidays_df["Value"] = list(holidays_ser.values())
holidays_df["Value"] = holidays_df["Value"].apply(lambda x: "{0:.0%}".format(x))
holidays_df.sort_values("Holiday", inplace=True)
yearly_seasonality = (
    yearly_seasonality[["yearly", "day"]].groupby("day").mean().reset_index()
)
yearly_seasonality.rename(
    {"day": "Day of year", "yearly": "Value"}, axis=1, inplace=True
)
dayly_seasonalities.rename({"day": "Day type"}, axis=1, inplace=True)
weekly_seasonality.rename({"weekly": "Value"}, axis=1, inplace=True)
weekly_seasonality["Day of week"] = weekly_seasonality["day"].map(DAY_OF_WEEK)
weekly_seasonality = (
    weekly_seasonality[["Value", "Day of week", "day"]]
    .groupby(["Day of week", "day"])
    .mean()
    .reset_index()
    .sort_values("day")
)

2022-04-17 14:49:14,336 - kedro.io.data_catalog - INFO - Loading data from `forecast_system` (ParquetDataSet)...
2022-04-17 14:49:14,379 - kedro.io.data_catalog - INFO - Loading data from `dayly_seasonalities_system` (ParquetDataSet)...
2022-04-17 14:49:14,383 - kedro.io.data_catalog - INFO - Loading data from `weekly_seasonality_system` (ParquetDataSet)...
2022-04-17 14:49:14,386 - kedro.io.data_catalog - INFO - Loading data from `yearly_seasonality_system` (ParquetDataSet)...
2022-04-17 14:49:14,389 - kedro.io.data_catalog - INFO - Loading data from `holidays_ser` (JSONDataSet)...


<IPython.core.display.Javascript object>

In [47]:
checklist = [
    "Yearly seasonality",
    "Weekly seasonality",
    "Dayly seasonalities",
    "Holidays",
]
date_range = [
    unixTimeMillis(pd.Timestamp(year=2020, month=1, day=1)),
    unixTimeMillis(pd.Timestamp(year=2022, month=2, day=1)),
]
demand_aggregation = "Dayly"

<IPython.core.display.Javascript object>

In [48]:
ini_date = pd.Timestamp(date_range[0], unit="s")
end_date = pd.Timestamp(date_range[1], unit="s")
cols_prediction = []
if "Yearly seasonality" in checklist:
    cols_prediction += ["yearly"]
if "Weekly seasonality" in checklist:
    cols_prediction += ["weekly"]
if "Dayly seasonalities" in checklist:
    cols_prediction += ["weekday", "saturday", "sunday", "holiday"]
if "Holidays" in checklist:
    cols_prediction += ["holidays"]
forecast_df["yhat"] = 1
for c in cols_prediction:
    forecast_df["yhat"] = forecast_df["yhat"] + forecast_df[c]
forecast_df["yhat"] = (forecast_df["trend"] * forecast_df["yhat"]).apply(
    lambda x: max(x, 0)
)
main_fig_df = forecast_df[["ds", "y", "yhat"]].copy()
main_fig_df.rename(
    {"ds": "Date", "y": "Observed", "yhat": "Modelled"}, axis=1, inplace=True
)
if demand_aggregation == "Dayly":
    main_fig_df["Date"] = main_fig_df["Date"].apply(lambda x: x.date())
    main_fig_df = main_fig_df.groupby("Date").sum().reset_index()
if demand_aggregation == "Monthly":
    main_fig_df["Date"] = main_fig_df["Date"].apply(
        lambda x: pd.Timestamp(year=x.year, month=x.month, day=1)
    )
    main_fig_df = main_fig_df.groupby("Date").sum().reset_index()
main_fig_df = main_fig_df[
    (main_fig_df["Date"] >= ini_date) & (main_fig_df["Date"] <= end_date)
].copy()
main_fig_df_final = main_fig_df.melt(
    id_vars=["Date"], var_name="Type", value_name="Demand"
)
figure = px.line(
    main_fig_df_final,
    x="Date",
    y="Demand",
    color="Type",
    title="Observed and modelled demand",
)
figure.update_layout(**LAYOUT_GRAPHS)
figure.update_xaxes(**XAXES_CONFIG)
figure.update_yaxes(**YAXES_CONFIG)

mets = pd.DataFrame(columns=["Metric", "Value", "Pre-COVID", "COVID"])

/opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



<IPython.core.display.Javascript object>

In [49]:
main_fig_df.reset_index(inplace=True, drop=True)
covid_date = pd.Timestamp(year=2020, month=3, day=20)
main_fig_df_precov = main_fig_df[pd.to_datetime(main_fig_df["Date"]) <= covid_date]
main_fig_df_cov = main_fig_df[pd.to_datetime(main_fig_df["Date"]) > covid_date]
for i, met in enumerate(
    [
        metrics.mean_absolute_error,
        metrics.mean_absolute_percentage_error,
        metrics.mean_squared_error,
    ]
):

    mets.loc[i] = [
        met.__name__,
        met(main_fig_df["Observed"], main_fig_df["Modelled"]),
        met(main_fig_df_precov["Observed"], main_fig_df_precov["Modelled"])
        if len(main_fig_df_precov) > 0
        else "NA",
        met(main_fig_df_cov["Observed"], main_fig_df_cov["Modelled"])
        if len(main_fig_df_cov) > 0
        else "NA",
    ]
mets.iloc[1, 1:] = mets.iloc[1, 1:].map(lambda x: "{0:.0%}".format(x))
mets.iloc[0, 1:] = mets.iloc[0, 1:].map(lambda x: int(round(x, -3)))
mets.iloc[2, 1:] = mets.iloc[2, 1:].map(lambda x: int(round(x ** 0.5, -3)))
mets.iloc[2, 0] = "root_mean_squared_error"

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
mets

,Metric,Value,Pre-COVID,COVID
0,mean_absolute_error,93000,135000,88000
1,mean_absolute_percentage_error,14%,13%,14%
2,root_mean_squared_error,134000,194000,125000


<IPython.core.display.Javascript object>